In [1]:
# !pip install numpy
# !pip install pandas
# !pip install nltk
# !pip install plotly
# !pip install matplotlib
# !pip install scikit-learn
# !pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
import numpy as np
import pandas as pd
import nltk
#from sklearn.feature_extraction import stop_words
from nltk.text import Text
from collections import Counter
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pandas import DataFrame
from plotly import graph_objs as go
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# WORKDIR = 'C:/Users/thewr/git/sentiment_analysis_twitter_netflix/'
WORKDIR = '/home/cesar/BD/sentiment_analysis_twitter_netflix/'

netflix_tweets_proc_file = WORKDIR + '/Data/Processed/netflix_all_tweets.parquet'

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 300)
from nltk.stem import RSLPStemmer





# Data reading 

In [8]:
df_netflix_all_tweets = pd.read_parquet(netflix_tweets_proc_file)

# print('shape:', df_netflix_all_tweets.shape)
# print('columns:', df_netflix_all_tweets.columns)

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [3]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0


In [4]:
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\thewr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\thewr\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [5]:
final_stopwords_list = nltk.corpus.stopwords.words('english') + nltk.corpus.stopwords.words('portuguese')

In [6]:
newStopWords = ['pra', 'diaz','juliette','lumena','carla','bbb21','gilberto','victor', 'hugo', 'globo', 'rodolfo', 'thais...','thaís','thais', 'caio', 'arcrebiano', 'brother',
               'lucas', 'bbb2021', 'paredão', 'probosta','projota','lider','líder', 'karol', 'conka', 'pocah', 'penteado', 'rodolffo','camilla','sarah','nego',
               'fiuk','fiukk','fiu','arthur','viih','boninho','big','pocahh','bbb']
final_stopwords_list.extend(newStopWords)

In [7]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0


# Models evaluation

## Sentilex

In [8]:
#Importing Sentilex lexicon with polarity words
sentilexpt = open('SentiLex-lem-PT02.txt')

#Creating dictionary with polarity words
dic_polarity_word = {}
for i in sentilexpt.readlines():
  pos_point = i.find('.')
  word = (i[:pos_point])
  pol_pos = i.find('POL')
  polarity = (i[pol_pos+7:pol_pos+9]).replace(';', '')
  dic_polarity_word[word] = polarity

#print (dic_polarity_word)

In [9]:
def sentiment_value_prediction(text):
    text = text.lower()
    l_sentiment = []    

    for p in text.split():    
        l_sentiment.append(int(dic_polarity_word.get(p, 0)))
    
    value = sum(l_sentiment)                    
    
    
    if value > 0:
        return 2.0
    elif value == 0:
        return 0.0
    elif value < 0:
        return 1.0


df_netflix_all_tweets['sentiment_sentilex']= df_netflix_all_tweets['text_x'].apply(lambda x:sentiment_value_prediction(x))

In [10]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento,sentiment_sentilex
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0,0.0
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0,0.0


In [11]:
#sentiment distribution according to Sentilex
plt.figure(figsize=(12,6))
ax = sns.countplot(x='sentiment_sentilex',palette=['gray','red','green'], data=df_netflix_all_tweets)
ax.set_xticklabels(['Neutral','Negative','Positive'], rotation='horizontal', fontsize=10)
ax.set_title('Sentiment Distribution - Sentilex Lexicon')

Text(0.5, 1.0, 'Sentiment Distribution - Sentilex Lexicon')

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(df_netflix_all_tweets['sentimento'], df_netflix_all_tweets['sentiment_sentilex'])

0.41909049977487617

In [13]:
print(metrics.classification_report(df_netflix_all_tweets['sentimento'], df_netflix_all_tweets['sentiment_sentilex']))

              precision    recall  f1-score   support

         0.0       0.42      0.90      0.57      9019
         1.0       0.29      0.28      0.29      1627
         2.0       0.64      0.06      0.11     11564

    accuracy                           0.42     22210
   macro avg       0.45      0.41      0.32     22210
weighted avg       0.52      0.42      0.31     22210



In [14]:
print (pd.crosstab(df_netflix_all_tweets['sentimento'], df_netflix_all_tweets['sentiment_sentilex'], rownames=['Real'], colnames=['Predito'], margins=True))

Predito    0.0   1.0   2.0    All
Real                             
0.0       8135   560   324   9019
1.0       1087   457    83   1627
2.0      10304   544   716  11564
All      19526  1561  1123  22210


## Leia

In [15]:
#LeIA is a fork from tool and lexicon VADER (Valence Aware Dictionary and sEntiment Reasoner) adapted to portuguese texts
#https://github.com/rafjaa/LeIA
import leia
from leia import SentimentIntensityAnalyzer

def sentiment_prediction_leia(text):
    
    s = SentimentIntensityAnalyzer()
    
    polarity = s.polarity_scores(text)
    return polarity
   

df_netflix_all_tweets['sentiment_leia_compound']= df_netflix_all_tweets['text_cleaned'].apply(lambda x:sentiment_prediction_leia(x))

In [16]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento,sentiment_sentilex,sentiment_leia_compound
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}"


In [17]:
#extractig only de compound value from 'sentiment_leia_compound' column
df_netflix_all_tweets['sentiment_leia_number'] = df_netflix_all_tweets['sentiment_leia_compound'].apply(pd.Series)['compound']

In [18]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento,sentiment_sentilex,sentiment_leia_compound,sentiment_leia_number
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0


In [19]:
#adapting leIa sentiment number to compare with 'sentimento' column
df_netflix_all_tweets['sentiment_leia_number_to_compare_metrics'] = df_netflix_all_tweets['sentiment_leia_number'].apply(lambda x: 2.0 if x > 0 else 1.0 if x < 0 else 0.0)

In [20]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento,sentiment_sentilex,sentiment_leia_compound,sentiment_leia_number,sentiment_leia_number_to_compare_metrics
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0,0.0
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0,0.0,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0,0.0


In [21]:
#removing columns of no use
df_netflix_all_tweets.drop(('sentiment_leia_compound'),axis=1, inplace=True)

In [22]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento,sentiment_sentilex,sentiment_leia_number,sentiment_leia_number_to_compare_metrics
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0,0.0,0.0,0.0
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0,0.0,0.0,0.0


In [23]:
#sentiment distribution according to LeIa
plt.figure(figsize=(12,6))
ax = sns.countplot(x='sentiment_leia_number_to_compare_metrics',palette=['gray','red','green'],data=df_netflix_all_tweets)
ax.set_xticklabels(['Neutral','Negative','Positive'], rotation='horizontal', fontsize=10)
ax.set_title('Sentiment Distribution - Leia Lexicon')

Text(0.5, 1.0, 'Sentiment Distribution - Leia Lexicon')

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(df_netflix_all_tweets['sentimento'], df_netflix_all_tweets['sentiment_leia_number_to_compare_metrics'])

0.4115263394867177

In [25]:
print(metrics.classification_report(df_netflix_all_tweets['sentimento'], df_netflix_all_tweets['sentiment_leia_number_to_compare_metrics']))

              precision    recall  f1-score   support

         0.0       0.42      0.64      0.51      9019
         1.0       0.20      0.53      0.29      1627
         2.0       0.60      0.22      0.32     11564

    accuracy                           0.41     22210
   macro avg       0.41      0.46      0.37     22210
weighted avg       0.50      0.41      0.39     22210



In [26]:
print (pd.crosstab(df_netflix_all_tweets['sentimento'], df_netflix_all_tweets['sentiment_leia_number_to_compare_metrics'], rownames=['Real'], colnames=['Predito'], margins=True))

Predito    0.0   1.0   2.0    All
Real                             
0.0       5777  1798  1444   9019
1.0        526   868   233   1627
2.0       7320  1749  2495  11564
All      13623  4415  4172  22210


In [27]:
#Preparing data to apply other classification algorithms
stemmer_ptbr = RSLPStemmer()
def stemming_tokenizer(raw_text):     
    words = raw_text.split() 
    words = [word for word in words if word not in final_stopwords_list]
    words = [stemmer_ptbr.stem(word) for word in words]
    return words  

In [28]:
df_netflix_all_tweets.head(2)

,reaction_to_status_id,reaction_to_status_text,reaction_to_status_created_at,id,text_x,created_at,url,text_cleaned,in_reply_to_status_id_str,quoted_status_id_str,sentimento,sentiment_sentilex,sentiment_leia_number,sentiment_leia_number_to_compare_metrics
0,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229083223515136,@NetflixBrasil oie https://t.co/33L2geGxWD,Mon Feb 01 13:13:05 +0000 2021,NaN,mencao_arroba oie,1356229042647789569,None,0.0,0.0,0.0,0.0
1,1356229042647789569,"Em fevereiro não posso, tô com a agenda cheia. 👇🏾 https://t.co/IBI89vqg5H",Mon Feb 01 13:12:55 +0000 2021,1356229130489126920,@NetflixBrasil Já ia falar que vcs estavam atrasados 😂,Mon Feb 01 13:13:16 +0000 2021,NaN,mencao_arroba ja ia falar que vcs estavam atrasados rosto_chorando_de_rir,1356229042647789569,None,2.0,0.0,0.0,0.0


In [29]:
y = df_netflix_all_tweets['sentimento']

In [30]:
vectorizer_tfidf = TfidfVectorizer(tokenizer=stemming_tokenizer, min_df=0.001, max_df=0.99)
X = vectorizer_tfidf.fit_transform(df_netflix_all_tweets['text_cleaned'])
X.shape

(22210, 634)

In [31]:
vocab = np.array(vectorizer_tfidf.get_feature_names())

C:\Users\thewr\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [32]:
#splitting dataframe into training and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=11)

## Randon Forest

In [33]:
from sklearn.ensemble import RandomForestClassifier
m = RandomForestClassifier(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)

RandomForestClassifier(max_features=0.5, min_samples_leaf=3, n_estimators=40,
                       n_jobs=-1, oob_score=True)

In [34]:
predicted = m.predict(X_test)

In [35]:
predicted

array([2., 2., 0., ..., 2., 0., 2.])

In [36]:
# importances = m.feature_importances_
# sem_importancia = []
# for i,w in enumerate(importances):
#   if w == 0:
#     sem_importancia.append(vocab[i])
#     print(vocab[i],w)

In [37]:
# importances = m.feature_importances_
# c_importancia = []
# for i,w in enumerate(importances):
#   if w > 0:
#     c_importancia.append(vocab[i])
#     print(vocab[i],w)

In [38]:
#len(c_importancia)

In [39]:
#len(sem_importancia)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicted)

0.7926330150068213

In [41]:
print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

         0.0       0.71      0.87      0.78      2995
         1.0       0.78      0.42      0.55       555
         2.0       0.89      0.78      0.83      3780

    accuracy                           0.79      7330
   macro avg       0.79      0.69      0.72      7330
weighted avg       0.81      0.79      0.79      7330



In [42]:
print (pd.crosstab(y_test, predicted, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0.0  1.0   2.0   All
Real                          
0.0      2615   48   332  2995
1.0       279  235    41   555
2.0       801   19  2960  3780
All      3695  302  3333  7330


## SVM

In [43]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test)

In [44]:
# model accuracy for X_test   
accuracy = svm_model_linear.score(X_test, y_test) 

In [45]:
print(accuracy)

0.8004092769440655


In [46]:
print(metrics.classification_report(y_test, svm_predictions))

              precision    recall  f1-score   support

         0.0       0.70      0.92      0.79      2995
         1.0       0.82      0.38      0.52       555
         2.0       0.92      0.77      0.84      3780

    accuracy                           0.80      7330
   macro avg       0.82      0.69      0.72      7330
weighted avg       0.82      0.80      0.80      7330



In [47]:
print (pd.crosstab(y_test, svm_predictions, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0.0  1.0   2.0   All
Real                          
0.0      2741   31   223  2995
1.0       314  210    31   555
2.0       850   14  2916  3780
All      3905  255  3170  7330


## KNN

In [48]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 7).fit(X_train, y_train) 
knn_predictions = knn.predict(X_test)
# accuracy on X_test 
accuracy = knn.score(X_test, y_test) 
print (accuracy)

0.7163710777626193


In [49]:
print(metrics.classification_report(y_test, knn_predictions))

              precision    recall  f1-score   support

         0.0       0.64      0.79      0.71      2995
         1.0       0.78      0.24      0.37       555
         2.0       0.80      0.73      0.76      3780

    accuracy                           0.72      7330
   macro avg       0.74      0.59      0.61      7330
weighted avg       0.73      0.72      0.71      7330



In [50]:
print (pd.crosstab(y_test, knn_predictions, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0.0  1.0   2.0   All
Real                          
0.0      2366   26   603  2995
1.0       331  133    91   555
2.0      1016   12  2752  3780
All      3713  171  3446  7330


## Decision Tree Classifier

In [51]:
# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(max_depth = 2).fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
  


In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, dtree_predictions)

0.6244201909959072

In [53]:
print(metrics.classification_report(y_test, dtree_predictions))

              precision    recall  f1-score   support

         0.0       0.52      0.99      0.68      2995
         1.0       0.00      0.00      0.00       555
         2.0       0.99      0.42      0.59      3780

    accuracy                           0.62      7330
   macro avg       0.50      0.47      0.43      7330
weighted avg       0.72      0.62      0.59      7330



C:\Users\thewr\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thewr\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\thewr\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
print (pd.crosstab(y_test, dtree_predictions, rownames=['Real'], colnames=['Predito'], margins=True))

Predito   0.0   2.0   All
Real                     
0.0      2979    16  2995
1.0       555     0   555
2.0      2182  1598  3780
All      5716  1614  7330


## Naive Bayes

In [55]:
from sklearn.naive_bayes import GaussianNB 
gnb = GaussianNB().fit(X_train.toarray(), y_train) 
gnb_predictions = gnb.predict(X_test.toarray()) 
  
# accuracy on X_test 
accuracy = gnb.score(X_test.toarray(), y_test) 
print (accuracy)

0.40204638472032744


In [56]:
print(metrics.classification_report(y_test, gnb_predictions))

              precision    recall  f1-score   support

         0.0       0.55      0.13      0.21      2995
         1.0       0.12      0.89      0.20       555
         2.0       0.88      0.54      0.67      3780

    accuracy                           0.40      7330
   macro avg       0.52      0.52      0.36      7330
weighted avg       0.69      0.40      0.45      7330



In [57]:
print (pd.crosstab(y_test, gnb_predictions, rownames=['Real'], colnames=['Predito'], margins=True))

Predito  0.0   1.0   2.0   All
Real                          
0.0      396  2326   273  2995
1.0       49   492    14   555
2.0      269  1452  2059  3780
All      714  4270  2346  7330


# Conclusion

## SVM seems to be the best prediction algorithm with an acceptable accuracy of 0.81 